In [7]:
import pandas as pd
import requests

In [8]:
def get_weather_features(lat, lon, start_date, end_date):
    """
    Fetch and summarize weather data for a given location and date range.
    Returns a single-row DataFrame with features like temp, rainfall, humidity.
    """

    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "pressure_msl",
            "precipitation",
            "snowfall",
            "cloud_cover",
            "cloud_cover_high",
            "cloud_cover_mid",
            "cloud_cover_low",
            "shortwave_radiation",
            "wind_speed_10m",
            "wind_direction_10m",
            "wind_gusts_10m"
        ],
        "timezone": "auto"
    }

    response = requests.get(url, params=params)
    data = response.json()

    if 'hourly' not in data:
        print("⚠️ Weather data not available for this location/date.")
        return pd.DataFrame()

    df = pd.DataFrame(data['hourly'])

    # Aggregate features over the season
    features = {
        "temperature_2m": df["temperature_2m"].mean(),
        "relative_humidity_2m": df["relative_humidity_2m"].mean(),
        "pressure_msl": df["pressure_msl"].mean(),
        "precipitation": df["precipitation"].mean(),
        "snowfall": df["snowfall"].mean(),
        "cloud_cover": df["cloud_cover"].mean(),
        "cloud_cover_high": df["cloud_cover_high"].mean(),
        "cloud_cover_mid": df["cloud_cover_mid"].mean(),
        "cloud_cover_low": df["cloud_cover_low"].mean(),
        "shortwave_radiation": df["shortwave_radiation"].mean(),
        "wind_speed_10m": df["wind_speed_10m"].mean(),
        "wind_direction_10m": df["wind_direction_10m"].mean(),
        "wind_gusts_10m": df["wind_gusts_10m"].mean()
    }

    return pd.DataFrame([features])


In [11]:
latitude = 19.997454
longitude = 73.789803
start_date = "2025-09-24"
end_date = "2025-09-25"
get_weather_features(latitude, longitude, start_date, end_date)

,temperature_2m,relative_humidity_2m,pressure_msl,precipitation,snowfall,cloud_cover,cloud_cover_high,cloud_cover_mid,cloud_cover_low,shortwave_radiation,wind_speed_10m,wind_direction_10m,wind_gusts_10m
0,22.823333,87.266667,1008.94,0.05,0.0,77.666667,51.633333,18.7,36.466667,190.066667,10.573333,265.733333,23.62


In [8]:
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
    "latitude": 19.997454, 
    "longitude": 73.789803,
    "start_date": "2025-09-24",
    "end_date": "2025-09-25",
    "hourly": [
        "temperature_2m",
        "relative_humidity_2m",
        "pressure_msl",
        "precipitation",
        "snowfall",
        "cloud_cover",
        "cloud_cover_high",
        "cloud_cover_mid",
        "cloud_cover_low",
        "shortwave_radiation",
        "wind_speed_10m",
        "wind_direction_10m",
        "wind_speed_80m",
        "wind_direction_80m",
        "wind_gusts_10m"
    ],
    "timezone": "auto"
}

response = requests.get(url, params=params)
data = response.json()
data

{'latitude': 19.999998,
 'longitude': 73.8,
 'generationtime_ms': 0.8661746978759766,
 'utc_offset_seconds': 19800,
 'timezone': 'Asia/Kolkata',
 'timezone_abbreviation': 'GMT+5:30',
 'elevation': 583.0,
 'hourly_units': {'time': 'iso8601',
  'temperature_2m': '°C',
  'relative_humidity_2m': '%',
  'pressure_msl': 'hPa',
  'precipitation': 'mm',
  'snowfall': 'cm',
  'cloud_cover': '%',
  'cloud_cover_high': '%',
  'cloud_cover_mid': '%',
  'cloud_cover_low': '%',
  'shortwave_radiation': 'W/m²',
  'wind_speed_10m': 'km/h',
  'wind_direction_10m': '°',
  'wind_speed_80m': 'undefined',
  'wind_direction_80m': 'undefined',
  'wind_gusts_10m': 'km/h'},
 'hourly': {'time': ['2025-09-24T00:00',
   '2025-09-24T01:00',
   '2025-09-24T02:00',
   '2025-09-24T03:00',
   '2025-09-24T04:00',
   '2025-09-24T05:00',
   '2025-09-24T06:00',
   '2025-09-24T07:00',
   '2025-09-24T08:00',
   '2025-09-24T09:00',
   '2025-09-24T10:00',
   '2025-09-24T11:00',
   '2025-09-24T12:00',
   '2025-09-24T13:00',
  

In [3]:
import pvlib
from pvlib.location import Location

In [11]:
def get_solar_feature(latitude, longitude, timezone="Asia/Kolkata", panel_tilt = 30, panel_azimuth=180):
    site = Location(latitude, longitude, timezone)
    times = pd.date_range(start=pd.Timestamp.now(tz=timezone), periods=24, freq='H')
    solar_position = site.get_solarposition(times)

    azimuth = solar_position['azimuth']
    zenith = solar_position['zenith']
    aoi = pvlib.irradiance.aoi(
        surface_tilt=panel_tilt,
        surface_azimuth=panel_azimuth,
        solar_zenith=zenith,
        solar_azimuth=azimuth
    )

    return pd.DataFrame({
        'angle_of_incidence': [aoi.mean()],
        'zenith': [zenith.mean()],
        'azimuth': [azimuth.mean()]
        })

In [5]:
latitude = 19.997454
longitude = 73.789803
start_date = "2025-09-24"
end_date = "2025-09-25"
get_solar_feature(latitude, longitude)

C:\Users\Dhiraj\AppData\Local\Temp\ipykernel_20232\847245866.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  times = pd.date_range(start=pd.Timestamp.now(tz=timezone), periods=24, freq='H')


,angle_of_incidence,azimuth,zenith
0,89.428734,180.802777,90.857193


In [12]:
weather_df = get_weather_features(latitude, longitude, start_date, end_date)
solar_df = get_solar_feature(latitude, longitude, panel_tilt=30, panel_azimuth=180)
combined_df = pd.concat([weather_df, solar_df], axis=1)
combined_df

C:\Users\Dhiraj\AppData\Local\Temp\ipykernel_20232\2720064764.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  times = pd.date_range(start=pd.Timestamp.now(tz=timezone), periods=24, freq='H')


,temperature_2m,relative_humidity_2m,pressure_msl,precipitation,snowfall,cloud_cover,cloud_cover_high,cloud_cover_mid,cloud_cover_low,shortwave_radiation,wind_speed_10m,wind_direction_10m,wind_gusts_10m,angle_of_incidence,zenith,azimuth
0,22.823333,87.266667,1008.94,0.05,0.0,77.666667,51.633333,18.7,36.466667,190.066667,10.573333,265.733333,23.62,89.4274,90.858496,182.065795


In [13]:
from datetime import datetime, timedelta

def get_default_dates():
    # Get today's date in yyyy-mm-dd format
    today = datetime.now()

    # Define start_date = 2 days ago
    start_date = (today - timedelta(days=2)).strftime('%Y-%m-%d')

    # Define end_date = yesterday (full day)
    end_date = (today - timedelta(days=1)).strftime('%Y-%m-%d')

    return start_date, end_date

# Usage
start_date, end_date = get_default_dates()
print(start_date)
print(end_date)

2025-09-24
2025-09-25
